# scqubits example: composite Hilbert spaces
J. Koch and P. Groszkowski<br>
E. Blackwell

For further documentation of scqubits see https://scqubits.readthedocs.io/en/latest/.

---

In [1]:
import scqubits as scq

# Working with composite Hilbert spaces and interfacing with QuTiP

Systems of interest for quantum information processing will involve multiple qubits as well as oscillators with mutual coupling. The resulting Hilbert space is the tensor product of the individual constituent Hilbert spaces. The `HilbertSpace` class allows one to define such coupled systems, to define the interactions between them, and to contruct the overall Hamiltonian. From this, dressed eigenenergies and eigenstates can be extracted. The operator matrices and state vectors  at the `HilbertSpace` level are given as QuTiP `Qobj` instances. This interface to QuTiP is particularly helpful if the task at hand is the simulation of time-dependent dynamics of the coupled system, perhaps including additional drive terms.

## Example: two ocilllator qubits coupled by an rf SQUID

As an interesting example of a coupled quantum system consider two harmonic modes coupled by an rf SQUID. The effective Hamiltonian describing this composite system, after integrating out the SQUID degrees of freedom,  is given by [https://www.nature.com/articles/s41567-019-0703-5]

$\displaystyle H/\hbar = \omega_a a^\dagger a + \omega_b b^\dagger b + g(a^\dagger b + a b^\dagger) + g_2(a^\dagger a^\dagger b + a a b^\dagger) + \chi_{ab} a^\dagger ab^\dagger b + \frac{\chi_{aa}}{2} a^\dagger a^\dagger a a + \frac{\chi_{bb}}{2} b^\dagger b^\dagger b b$,

which can be rewritten as

$\displaystyle H = E_{\text{osc}_a} a^\dagger a + E_{\text{osc}_b} b^\dagger b + g(a^\dagger b + a b^\dagger) + g_2(a^\dagger a^\dagger b + a a b^\dagger) + \chi_{ab} a^\dagger ab^\dagger b - K_a a^\dagger a^\dagger a a - K_b b^\dagger b^\dagger b b$.

### Define Hilbert space components, initialize `HilbertSpace` object

To set up the Hilbert space, we define the separate components as Kerr oscillators and initialize a `HilbertSpace` object by submitting the list of all subsystems:

In [2]:
# Set up the components / subspaces of our Hilbert space
E_osc_a = 4.284
K_a = 0.003

E_osc_b = 7.073
K_b = 0.015

osc1 = scq.KerrOscillator(
    E_osc = E_osc_a,
    K = K_a,
    truncated_dim = 4,
)

osc2 = scq.KerrOscillator(
    E_osc = E_osc_b,
    K = K_b,
    truncated_dim = 4,
)

# Form a list of all components making up the Hilbert space.
hilbertspace = scq.HilbertSpace([osc1, osc2])

In [3]:
print(hilbertspace)

HilbertSpace:  subsystems
-------------------------

KerrOscillator------| [KerrOscillator_1]
                    | E_osc: 4.284
                    | K: 0.003
                    | l_osc: None
                    | truncated_dim: 4
                    |
                    | dim: 4


KerrOscillator------| [KerrOscillator_2]
                    | E_osc: 7.073
                    | K: 0.015
                    | l_osc: None
                    | truncated_dim: 4
                    |
                    | dim: 4




While we yet have to set up the interactions between the components, we can already obtain the bare Hamiltonian of the non-interacting subsystems, expressed as a matrix in the joint Hilbert space:

In [4]:
bare_hamiltonian = hilbertspace.bare_hamiltonian()
bare_hamiltonian

Quantum object: dims = [[4, 4], [4, 4]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     7.073  0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.    14.116  0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.    21.129  0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     4.284  0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.    11.357  0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.    18.4    0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.    25.413  0.     0.
   0.     0.     0.     0.     0.  

As we have not added any interactions yet, there is no difference between the methods `bare_hamiltonian()` and `hamiltonian()` at the moment:

In [5]:
bare_hamiltonian == hilbertspace.hamiltonian()

True

The Hamiltonian matrix is given in the form of a QuTiP quantum object.
This facilitates simple incorporation of Hamiltonians generated with `scqubits` into any of the solvers QuTiP offers for time evolution.

### Set up interaction terms between individual subsystems

Interactions are added using the `add_interaction()` method. If the interaction is of the form $g \cdot \text{op1} \cdot \text{op2}$, it can be added by specifying the coefficient $g$ and the two operators $\text{op1}$ and $\text{op2}$. Otherwise, the interaction is given as a Python string that is later evaluated.

1. $g(a^\dagger b + a b^\dagger)$
2. $g_2(a^\dagger a^\dagger b + a a b^\dagger)$
3. $\chi_{ab} a^\dagger ab^\dagger b$

In [6]:
# Option A: operator product specification via callables
# Term 1
g = 0.1

hilbertspace.add_interaction(
    g_strength = g,
    op1 = osc1.creation_operator,
    op2 = osc2.annihilation_operator,
    add_hc = True
)

# Option B: string-based specification of interaction term
# Term 2
g2 = 0.035 

hilbertspace.add_interaction(
    expr = "g2 * a.dag() * a.dag() * b",
    op1 = ("a", osc1.annihilation_operator),
    op2 = ("b", osc2.annihilation_operator),
    add_hc = True
)

# Term 3
chi_ab = 0.01

hilbertspace.add_interaction(
    expr = "chi_ab * a.dag() * a * b.dag() * b",
    op1 = ("a", osc1.annihilation_operator),
    op2 = ("b", osc2.annihilation_operator),
    add_hc = False
)

Now that the interactions are specified, the full Hamiltonian of the coupled system can be obtained with `hamiltonian()`:

In [7]:
dressed_hamiltonian = hilbertspace.hamiltonian()
dressed_hamiltonian

Quantum object: dims = [[4, 4], [4, 4]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          7.073       0.          0.          0.1         0.
   0.          0.          0.04949747  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.         14.116       0.          0.          0.14142136
   0.          0.          0.          0.07        0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         21.129       0.          0.
   0.17320508  0.          0.          0.          0.08573214  0.
   0.          0.          0.          0.        ]
 [ 0.          0.1         0.          0.          4.284       0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0. 

Since the composite Hamiltonian is a `qutip.Qobj`, the eigenvalues and eigenvectors can be now be obtained via the usual QuTiP routine:

In [8]:
evals, evecs = dressed_hamiltonian.eigenstates(eigvals=4)
print(evals)

[0.         4.28041835 7.07493032 8.55652435]


### GUI use for `HilbertSpace` object creation

As an alternative to the programmatic generation of a new `HilbertSpace` object, the following GUI-based creation process is supported:

In [9]:
hilbertspace_gui = scq.HilbertSpace.create()

Output()

In [10]:
print(hilbertspace_gui)

HilbertSpace:  subsystems
-------------------------

KerrOscillator------| [KerrOscillator_1]
                    | E_osc: 4.284
                    | K: 0.003
                    | l_osc: None
                    | truncated_dim: 4
                    |
                    | dim: 4


KerrOscillator------| [KerrOscillator_2]
                    | E_osc: 7.073
                    | K: 0.015
                    | l_osc: None
                    | truncated_dim: 4
                    |
                    | dim: 4



HilbertSpace:  interaction terms
--------------------------------
InteractionTerm----------| [InteractionTerm_0]
                         | expr: 'g * op1 * op2'
                         | operator_list: [(0, 'op1', array([[0.        , 0.        , 0.     ...
                         | const: {}
                         | add_hc: True

InteractionTerm----------| [InteractionTerm_1]
                         | expr: 'g2 * op1.dag() * op1.dag() * op2'
                         | o

In [11]:
hilbertspace_gui.hamiltonian()

Quantum object: dims = [[4, 4], [4, 4]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          7.073       0.          0.          0.1         0.
   0.          0.          0.04949747  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.         14.116       0.          0.          0.14142136
   0.          0.          0.          0.07        0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         21.129       0.          0.
   0.17320508  0.          0.          0.          0.08573214  0.
   0.          0.          0.          0.        ]
 [ 0.          0.1         0.          0.          4.284       0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0. 

We can see that it indeed corresponds to the Hamiltonian created programatically:

In [12]:
hilbertspace_gui.hamiltonian() == dressed_hamiltonian

True

### Spectrum lookup and converting between bare and dressed indices

In the dispersive regime where bare states don't hybridize too much, it may be useful to convert between dressed states and their closest product bare states as physical interpretation may be easier. To use lookup functions for state indices, energies and states, first generate the lookup table via `generate_lookup()`:

In [13]:
hilbertspace.generate_lookup()

We can first print the bare energies of the two Kerr oscillators:

In [14]:
print(hilbertspace.bare_eigenvals(osc1))
print(hilbertspace.bare_eigenvals(osc2))

[ 0.     4.284  8.562 12.834]
[ 0.     7.073 14.116 21.129]


Next, we can lookup the closest bare product state to the dressed state with index `8` with `bare_index()`, which corresponds to 1 excitation in the first Kerr oscillator and 2 excitations in the second.

In [15]:
hilbertspace.bare_index(8)

(1, 2)

Conversely, we can verify that the bare product state (1,2) most closely matches the dressed state with index `8`

In [16]:
hilbertspace.dressed_index((1,2))

8

Finally, we can get the eigenenergy for dressed index `8` using `energy_by_dressed_index()`:

In [17]:
hilbertspace.energy_by_dressed_index(8)

18.413688985491895

## Sweeping over an external parameter

Equipped with a programmatic interface to explore interactions between subsystems, scqubits provides the class `ParameterSweep` to facilitate computation of spectra as function of an external parameter. See the [example notebook](./demo_parametersweep.ipynb) for `ParameterSweep` to explore sweeps and visualizing sweep data.